In [5]:
import json
re_count = {}

# re2id
re2id = json.load(open("/nfs/turbo/umms-drjieliu/proj/prompt_re/KnowPrompt/dataset/biored_mapped/rel2id.json", "r", encoding='utf-8'))

with open("/nfs/turbo/umms-drjieliu/proj/prompt_re/KnowPrompt/dataset/biored_mapped/train.txt", "r", encoding='utf-8') as reader:
    all_lines = reader.readlines()
    for line in all_lines:
        ins = eval(line)
        relation = ins["relation"]
        if re2id[relation] not in re_count:
            re_count[re2id[relation]] = 1
        else:
            re_count[re2id[relation]] += 1

# sort by key
re_count = sorted(re_count.items(), key=lambda x: x[0])
re_count

[(0, 1332),
 (1, 1621),
 (2, 2761),
 (3, 118),
 (4, 1235),
 (5, 161),
 (6, 3),
 (7, 157),
 (8, 47)]

In [1]:
"RobertaForPrompt".rsplit(".", 1)

['RobertaForPrompt']

In [6]:
from transformers import AutoTokenizer
import transformers
import torch

In [8]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext")

In [2]:
entity_list = ["[object_start]", "[object_end]", "[subject_start]", "[subject_end]"]

In [6]:
tokenizer.add_special_tokens({'cls_token': "[CLS]"})

1

In [3]:
tokenizer.cls_token_id

2

In [5]:
tokenizer.__class__.__name__

'GPT2TokenizerFast'

In [5]:
# "roberta-large" from pretrained model
model = transformers.AutoModel.from_pretrained("microsoft/biogpt")

KeyError: 'biogpt'

In [4]:
# special tokens in tokenizer
tokenizer.all_special_tokens

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [58]:
tokenizer.mask_token

'<mask>'

In [56]:
tokenizer.get_vocab()['<mask>']

50264

In [63]:
# what token is input_ids 103
tokenizer.decode(0)

'<s>'

In [7]:
tokenizer.add_special_tokens({'additional_special_tokens': entity_list})

4

In [19]:
tokenizer.cls_token

'<s>'

In [20]:
[f"[T{i}]" for i in range(1,6)]

['[T1]', '[T2]', '[T3]', '[T4]', '[T5]']

In [23]:
# read lines /home/dengfy/KnowPrompt-master/dataset/semeval/test.txt
with open('/home/dengfy/KnowPrompt-master/dataset/semeval/test.txt', 'r') as f:
    lines = f.readlines()

In [35]:
example = eval(lines[0])
example

{'token': ['the',
  'most',
  'common',
  'audits',
  'were',
  'about',
  'waste',
  'and',
  'recycling',
  '.'],
 'h': {'name': 'audits', 'pos': [3, 4]},
 't': {'name': 'waste', 'pos': [6, 7]},
 'relation': 'Message-Topic(e1,e2)'}

In [60]:
SUBJECT = " ".join(example["token"][3: 4])
SUBJECT

'audits'

In [65]:
tokenizer(" "+SUBJECT,SUBJECT, add_special_tokens=True)

{'input_ids': [0, 26723, 2, 2, 5247, 2629, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [67]:
tokenizer.decode([0, 26723, 2, 2, 5247, 2629, 2])

'<s> audits</s></s>audits</s>'

In [1]:
import json
with open(f"dataset/semeval/rel2id.json", "r") as file:
    t = json.load(file)
    label_list = list(t)

In [4]:
from torch.nn.utils.rnn import pad_sequence

tokenizer = AutoTokenizer.from_pretrained("roberta-large")
def split_label_words(tokenizer, label_list):
    label_word_list = []
    for label in label_list:
        if label == 'no_relation' or label == "NA":
            label_word_id = tokenizer.encode('no relation', add_special_tokens=False)
            label_word_list.append(torch.tensor(label_word_id))
        else:
            tmps = label
            label = label.lower()
            label = label.split("(")[0]
            label = label.replace(":"," ").replace("_"," ").replace("per","person").replace("org","organization")
            label_word_id = tokenizer(label, add_special_tokens=False)['input_ids']
            print(label, label_word_id)
            label_word_list.append(torch.tensor(label_word_id))
    padded_label_word_list = pad_sequence([x for x in label_word_list], batch_first=True, padding_value=0)
    return padded_label_word_list

In [5]:
t = split_label_words(tokenizer, label_list)

component-whole [46362, 12, 11613, 4104]
other [7443]
instrument-agency [179, 41392, 12, 26904]
member-collection [8648, 12, 44443]
cause-effect [27037, 12, 26715]
entity-destination [46317, 12, 31549, 8111]
content-container [10166, 12, 46367]
message-topic [44773, 12, 45260]
product-producer [20565, 12, 26790, 7742]
member-collection [8648, 12, 44443]
entity-origin [46317, 12, 43211]
cause-effect [27037, 12, 26715]
component-whole [46362, 12, 11613, 4104]
message-topic [44773, 12, 45260]
product-producer [20565, 12, 26790, 7742]
entity-origin [46317, 12, 43211]
content-container [10166, 12, 46367]
instrument-agency [179, 41392, 12, 26904]
entity-destination [46317, 12, 31549, 8111]


In [6]:
t

tensor([[46362,    12, 11613,  4104],
        [ 7443,     0,     0,     0],
        [  179, 41392,    12, 26904],
        [ 8648,    12, 44443,     0],
        [27037,    12, 26715,     0],
        [46317,    12, 31549,  8111],
        [10166,    12, 46367,     0],
        [44773,    12, 45260,     0],
        [20565,    12, 26790,  7742],
        [ 8648,    12, 44443,     0],
        [46317,    12, 43211,     0],
        [27037,    12, 26715,     0],
        [46362,    12, 11613,  4104],
        [44773,    12, 45260,     0],
        [20565,    12, 26790,  7742],
        [46317,    12, 43211,     0],
        [10166,    12, 46367,     0],
        [  179, 41392,    12, 26904],
        [46317,    12, 31549,  8111]])

In [8]:
t.shape

torch.Size([19, 4])

In [19]:
tokenizer("[class1]", add_special_tokens=False)

{'input_ids': [10975, 4684, 134, 742], 'attention_mask': [1, 1, 1, 1]}

In [20]:
tokenizer.decode(10975)

'['

In [11]:
class_list = [f"[class{i}]" for i in range(1, 19+1)]
tokenizer.add_special_tokens({'additional_special_tokens': class_list})

19

In [13]:
[a[0] for a in tokenizer([f"[class{i}]" for i in range(1, 19+1)], add_special_tokens=False)['input_ids']]

[50265,
 50266,
 50267,
 50268,
 50269,
 50270,
 50271,
 50272,
 50273,
 50274,
 50275,
 50276,
 50277,
 50278,
 50279,
 50280,
 50281,
 50282,
 50283]